In [1]:
# 기본적인 라이브러리(기초세션에서 다룸)
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 구글 드라이브 연동

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 문제 1 : Classification 문제

해당 코드의 빈 곳(??)을 찾아 완성해주세요.

이전 '[0131]_LinearRegression_and_SVM_HW' 예제의 bike.csv에서 y값을 precipitation(범주형)으로 하는 classification문제를 해결해보자.


서울시 마포구의 날짜별, 시간별 기상상황과 따릉이 대여 수 데이터
즉, 따릉이 보관소별로 매일 자전거 대여

* id : 마포구에 있는 따릉이 보관소의 고유 id
* hour : 따릉이 보관소에서 기상상황을 측정한 시간
* temperature : 기온
* precipitation : 비가 오지 않았으면 0, 비가 오면 1, null은 비가 온 것도 아니고 안 온 것도 아니라서 센서가 확실히 측정불가한 상태
* windspeed : 풍속(평균)
* humidity : 습도
* visibility : 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
* ozone : 오존
* pm10 : 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
* pm2.5 : 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
* count : 측정한 날짜의 따릉이 대여 수

> 데이터 출처 : https://www.dacon.io/competitions/open/235576/data

## 데이터 전처리

In [7]:
# 데이터 불러오기

path = '/content/drive/MyDrive/DSL/2023-1 과제/[0131][HW]_LinearRegression_and_SVM/data/bike.csv'
bike = pd.read_csv(path)
bike.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431


In [9]:
# 결측치 처리 -> null이 포함된 행(데이터)은 삭제

bike.dropna(inplace = True)
bike.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [ ]:
# 범주형 X 데이터 one-hot encoding
# 유일하게 범주형 데이터였던, 'hour_bef_precipitation'열이 y값이 되었음으로
# 남은 X 열들(features)은 모두 연속형 데이터임으로, 따로 'one-hot encoding'을 해줄 필요가 없다.

In [10]:
# 'y_label', 'X_features' 만들기

y_label = bike.loc[:,'hour_bef_precipitation']
X_features = bike.drop(['hour_bef_precipitation'], axis=1)

In [11]:
# 다중공선성 고려 (1)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X_features.values, i) for i in range(X_features.shape[1])] # 리스트내포

vif

,features,VIF Factor
0,id,4.022339
1,hour,7.236586
2,hour_bef_temperature,18.456758
3,hour_bef_windspeed,6.963252
4,hour_bef_humidity,5.711342
5,hour_bef_visibility,7.199585
6,hour_bef_ozone,8.904015
7,hour_bef_pm10,5.860971
8,hour_bef_pm2.5,8.178193
9,count,6.616391


In [13]:
# vif가 가장 큰 ??열 제거

X2_features = X_features.drop(columns = ['hour_bef_temperature'], axis=1)

In [14]:
# 다중공선성 고려 (2)
# 가장 vif가 큰 값을 제거하여 다시 vif확인

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X2_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X2_features.values, i) for i in range(X2_features.shape[1])] # 리스트내포

vif

# vif값이 10을 넘지 않음으로, 다중공선성을 걱정하여 열을 제거할 필요가 없다.

,features,VIF Factor
0,id,4.013247
1,hour,7.236170
2,hour_bef_windspeed,6.957653
3,hour_bef_humidity,5.466206
4,hour_bef_visibility,5.987482
5,hour_bef_ozone,7.858466
6,hour_bef_pm10,5.771248
7,hour_bef_pm2.5,7.917995
8,count,5.654700


In [25]:
# X, y 를 8: 2로 데이터 분할
# stratify를 통해 y값 클래스 비율 일정하게 분할; 분류일 경우, 데이터분할시에 주의가 필요하다.
# random_state = 1로 지정

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X2_features, y_label, train_size=0.8,
                                                    test_size=0.2, random_state=1, stratify = y_label)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1056, 9) (265, 9) (1056,) (265,)


제일 위의 '중요사항'에 따라 이번시간에 다루는 모델을 사용할 때, 스케일링이 필요없다.

## 모델 학습

사용할 모델
* DecisionTreeClassifier : DecisionTree(의사결정나무) <- (1)
* Bagging : RandomForestClassifier <- (2)
* Boosting
  * XGBoost : XGBClassifier <- (3)
  * LightGBM : LGBClassifier <- (4)


학습방법
* 모델을 하이퍼라미터 값을 변경해가면서 10-Fold를 통해 학습데이터 평가. 즉, 학습데이터를 학습과 검증데이터로 쪼개는 과정을 반복하여 학습시키는 방법
  * 참고 : https://huidea.tistory.com/30

* DecisionTree 하이퍼파라미터 후보 : criterion : 'gini', 'entropy' , max_depth=5,10,15,20,None
* Random Forest 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , random_state=1

* XGBoost 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , gamma = 0.5,1,2  , random_state=1 

* LightGBM 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , random_state=1

* 하이퍼파라미터 선택을 위한 평가지표 : F1-Score



> 하이퍼파라미터의 종류가 많음으로, cross_val_score()보다 GridSearchCV()를 사용하자.
> * 해당 함수들에 대한 정보를 확인하기 위해 반드시 아래 문서 확인
>   * https://techblog-history-younghunjo1.tistory.com/100

> 각 모델의 하이퍼파라미터에 대한 정보는 sklearn문서를 참고
> * ex. (DecisionTree) https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [30]:
from sklearn.model_selection import GridSearchCV

참고사항

* 그냥 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = test_X, test_y
* k-fold 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = train_X, train_y
* k-fold를 사용하는 이유
  * 데이터의 수가 너무 적어, test_X, test_y를 만들기 어려운 경우
  * 하이퍼파라미터를 찾을 시에, test_X, test_y을 잘 예측하는 모델이 아닌 train_X, train_y을 잘 예측하는 모델로 하여, 과적합을 막기 위함

### 1. DecisionTree

In [31]:
from sklearn.tree import DecisionTreeClassifier

# 모델 객체 선언
clf_tree = DecisionTreeClassifier() # 분류모델

# train 데이터에 대해서 학습진행
clf_tree.fit(train_X, train_y)

# HyperParameter 후보군; 딕셔너리 형태로 저장
parameters={'criterion':['gini', 'entropy'],
            'max_depth':[5,10,15,20,None]}

# HyperParameter를 Tuning
model_dt=GridSearchCV(estimator=clf_tree, param_grid=parameters,
                     scoring='f1', cv=10, refit=True)

model_dt.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15, 20, None]},
             scoring='f1')

In [32]:
model_dt.best_params_

{'criterion': 'gini', 'max_depth': 15}

### 2. RandomForestClassifier

In [36]:
from sklearn.ensemble import RandomForestClassifier

# 객체 선언
randomforest=RandomForestClassifier() # 분류모델

# train 데이터에 대해서 학습진행
randomforest.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'n_estimators':[100, 200, 300],
            'max_depth':[3, 5, 10],
            'random_state':[1]}

# HyperParameter를 Tuning
model_rfc=GridSearchCV(estimator=randomforest, param_grid=parameters,
                     scoring='f1', cv=10, refit=True)

model_rfc.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [37]:
model_rfc.best_params_

{'max_depth': 5, 'n_estimators': 200, 'random_state': 1}

### 3. XGBClassifier

In [38]:
from xgboost import XGBClassifier # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
xgboost=XGBClassifier() # 분류모델

# train 데이터에 대해서 학습진행
xgboost.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'gamma': [0.5, 1, 2],
            'learning_rate': [0.001, 0.01, 0.1, 1],
            'max_depth': [3, 5, 10],
            'n_estimators': [100, 200, 300], 'random_state': [1]}

# HyperParameter를 Tuning
model_xgb=GridSearchCV(estimator=xgboost, param_grid=parameters,
                     scoring='f1', cv=10, refit=True)

model_xgb.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=XGBClassifier(),
             param_grid={'gamma': [0.5, 1, 2],
                         'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [39]:
model_xgb.best_params_

{'gamma': 2,
 'learning_rate': 0.001,
 'max_depth': 5,
 'n_estimators': 200,
 'random_state': 1}

### 4. LGBMClassifier

In [41]:
from lightgbm import LGBMClassifier # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
lightgbm=LGBMClassifier() # 회귀모델

# train 데이터에 대해서 학습진행
lightgbm.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'learning_rate': [0.001, 0.01, 0.1, 1],
            'max_depth': [3, 5, 10],
            'n_estimators': [100, 200, 300], 'random_state': [1]}

# HyperParameter를 Tuning
model_lgbm=GridSearchCV(estimator=lightgbm, param_grid=parameters,
                     scoring='f1', cv=10, refit=True)

model_lgbm.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [42]:
model_lgbm.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 1}

## 모델 평가

학습데이터로 평가했을 때 가장 좋은 성능을 보인 하이퍼 파라미터값의 DecisionTree, RandomForest, XGBoost, LightGBM모델을 테스트 데이터로 평가
* 서로 다른 분류모형을 비교할 때, 사용하는 평가지표 : Accuracy, Recall, Precision ,F1-Score

In [43]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

앞선 GridSearchCV(refit=True)를 하여 최적의 hyper-parameter에 대한 모델객체를 생성하고, model.fit(train_X)로 학습도 시켰기에 그 결과인 (model_dt, model_rfc, model_xgb, model_lgbm)를 따로 '객체선언,학습' 할 필요없이 바로 써주면 된다.

##### 일단 Train 데이터에 대해서 확인



In [44]:
# 'Train데이터 전부'에 대해 각 모델별 f1-score값 확인

print("DecisionTree model : {:.3f}".format(f1_score(train_y, model_dt.predict(train_X))))
print("RandomForest model : {:.3f}".format(f1_score(train_y, model_rfc.predict(train_X))))
print("XGBoost model : {:.3f}".format(f1_score(train_y, model_xgb.predict(train_X))))
print("LightGBM model : {:.3f}".format(f1_score(train_y, model_lgbm.predict(train_X))))

DecisionTree model : 1.000
RandomForest model : 0.836
XGBoost model : 0.691
LightGBM model : 1.000


F1-Score가 높을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. LightGBM
2. DecisionTree
3. RandomForest
4. XGBoost

In [45]:
# GridSearchCV()를 진행하여 교차검증을 수행할 때, '일부의 Train데이터인 Validation데이터'에 각 모델별 f1-score값 확인
# f1-score의 경우에 서로 다른 분류모델끼리 성능비교가 가능한 지표이다.

print("DecisionTree model : {:.3f}".format(model_dt.best_score_))
print("RandomForest model : {:.3f}".format(model_rfc.best_score_))
print("XGBoost model : {:.3f}".format(model_xgb.best_score_))
print("LightGBM model : {:.3f}".format(model_lgbm.best_score_))

DecisionTree model : 0.424
RandomForest model : 0.339
XGBoost model : 0.433
LightGBM model : 0.339


F1-Score가 높을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. XGBoost
2. DecisionTree
3. RandomForest, LightGBM

##### Test 데이터에 대해서 평가

In [46]:
# 'Test데이터'에 대해 각 모델별 accuracy_score값 확인

print("DecisionTree model : {:.3f}".format(accuracy_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(accuracy_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(accuracy_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(accuracy_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.955
RandomForest model : 0.981
XGBoost model : 0.981
LightGBM model : 0.974


In [47]:
# 'Test데이터'에 대해 각 모델별 recall_score 확인

print("DecisionTree model : {:.3f}".format(recall_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(recall_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(recall_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(recall_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.375
RandomForest model : 0.375
XGBoost model : 0.500
LightGBM model : 0.375


In [48]:
# 'Test데이터'에 대해 각 모델별 precision_score 확인

print("DecisionTree model : {:.3f}".format(precision_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(precision_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(precision_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(precision_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.300
RandomForest model : 1.000
XGBoost model : 0.800
LightGBM model : 0.600


In [49]:
# 'Test데이터'에 대해 각 모델별 f1_score 확인

print("DecisionTree model : {:.3f}".format(f1_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(f1_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(f1_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(f1_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.333
RandomForest model : 0.545
XGBoost model : 0.615
LightGBM model : 0.462


분류의 경우, 각 평가함수들은 목적에 따라 다르게 사용해야한다.

따라서 해당 문제의 경우, y값인 'hour_bef_precipitation'에서 비오는 것(1)과 비가 오지 않는 것(0) 중 어떤 것이 중요한지 생각하여 평가해야한다.

예를 들면,

Positive(1=비가옴)인 데이터를 Negative(0=비가안옴)로 잘못 판단하는 것이 큰 문제인 경우 재현율이 상대적으로 더 중요해지며, Negative(0=비가안옴)인 데이터를 Positive(1=비가옴)로 잘못 판단하는 것이 큰 문제인 경우 정밀도가더 중요해진다.

아래 글 참고
> https://ek-koh.github.io/data%20analysis/evaluation/

위 평가지표와 글을 보고 모델을 비교하여 서술

1. accuracy의 경우 주어진 task처럼 y의 분포가 불균형한 경우에는 적합한 지표가 아니다. 또한 accuracy를 기준으로 했을 때 모델 간에 큰 편차가 없다. <br>

2. recall은 실제 양성인 것 중에 모델이 양성이라고 잡아낸 비율을 말한다. XGB, LGBM = RF, DT 순으로 높았다. <br> y 분포가 불균형하기 때문에, accuracy가 높게 나타났었는데, recall에서는 실질적인 모델의 성능을 평가하여 그 점수가 상대적으로 낮아지는 것을 확인할 수 있다.

3. precision은 양성이라고 분류한 것 중에 실제 양성인 비율을 말한다. RF, XGB, LGBM, DT 순으로 높았다. 

4. f1 score는 recall과 precision의 조화평균으로, 두 지표를 종합적으로 고려한다. XGB, RF, LGBM, DT 순으로 높았다. <br> 

5. 종합하자면 XGB는 모든 지표에서 높은 점수를 고르게 받았다. 이는 XGM가 특별히 양성일 확률을 과다평가하지도 과소평가하지도 않은 가장 좋은 모델임을 의미한다. <br> RF의 경우 precision이 1이 나왔는데, 이는 모델이 거의 모든 데이터에 대해서 양성이라고 말했기 때문일 수도 있다. RF의 recall은 상대적으로 낮게 나왔다는 데에서 이를 확인할 수 있다. <br> LGBM은 RF와 유사하게 precision이 높게 나오는 문제를 가지고 있고 동시에 상대적으로 부정확하다. <br> 특별히 양성일 확률을 과대/과소평가하지는 않았지만 정확도가 낮아서 가장 열위인 모델로 드러났다.